# Data Scientist Challenge - LATAM Airlines

This is a notebook with the purpose of developing the solution to the Data Scientist challenge given by LATAM Airlines. 

It will be subdivided into the following segments, explaining in detail each one of them in their respective sections:


1.  Data Distribution - Exploratory Data Analysis
    - 1.1 Data Loading
    - 1.2 Data Distribution
    - 1.3 Conclusions about the distributions
2.  Feature Engineering
    - 2.1 Generating new features
    - 2.2 Exporting the features   
3.  Flight Delay Analysis
    - 3.1 Lateness rate and its relationship with other variables
    - 3.2 Analysis of the best flight delay predictors
4.  Model Training
5.  Model Evaluation
    - 5.1 Model Performance
    - 5.2 Metric Analysis
    - 5.3 Algorithm Analysis
    - 5.4 Feature Importance
    - 5.5 Further Improvements
